# Algo Project #

In [2]:
# Setup 
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas.io.data as web
%matplotlib inline

In [14]:
sp500_tickers_lil = ['AA','AAPL','ABC','ABT','ADBE','ADI','ADM','ADP','ADSK','AEE','AEP','AES']
sp500_tickers_joey = ['AA','AAPL','ABC','ABT','ADBE','ADI','ADM','ADP','ADSK','AEE','AEP','AES','AET','AFL','AGN','AIG','AIV','AIZ','AKAM','ALL','AMAT','AMGN','AMP','AMT','AMZN','AN','ANTM','AON','APA','APC','APD','AVB','AVY','AXP','AZO','BA','BAC','BAX','BBBY','BBT','BBY','BCR','BDX','BEN','BHI','BIIB','BK','BLL','BMY','BSX','BXP','C','CA','CAG','CAH','CAT','CB','CBG','CBS','CCE','CCL','CELG','CHK','CHRW','CI','CINF','CL','CLX','CMA','CMCSA','CME','CMI','CMS','CNP','COF','COH','COL','COP','COST','CPB','CSCO','CSX','CTAS','CTL','CTSH','CTXS','CVS','CVX','D','DD','DE','DFS','DGX','DHI','DHR','DIS','DOV','DOW','DRI','DTE','DUK','DVN','EA','EBAY','ECL','ED','EFX','EIX','EL','EMC','EMN','EMR','EOG','EQR','ESRX','ETFC','ETN','ETR','EXC','EXPD','EXPE','F','FCX','FDX','FE','FIS','FISV','FITB','FLR','FOXA','FTR','GAS','GD','GE','GILD','GIS','GLW','GOOGL','GPC','GPS','GS','GT','GWW','HAL','HAR','HAS','HBAN','HD','HES','HIG','HOG','HON','HOT','HPQ','HRB','HST','HSY','HUM','IBM','ICE','IFF','INTC','INTU','IP','IPG','ITW','JCI','JEC','JNJ','JNPR','JPM','JWN','K','KEY','KIM','KLAC','KMB','KO','KR','KSS','L','LB','LEG','LEN','LH','LLL','LLTC','LLY','LM','LMT','LNC','LOW','LUK','LUV','M','MAR','MAS','MAT','MCD','MCHP','MCK','MCO','MDLZ','MDT','MET','MHFI','MKC','MMC','MMM','MO','MON','MRK','MRO','MS','MSFT','MSI','MTB','MU','MUR','MYL','NBL','NEE','NEM','NI','NKE','NOC','NOV','NSC','NTAP','NTRS','NUE','NVDA','NWL','OMC','ORCL','OXY','PAYX','PBI','PCAR','PCG','PDCO','PEG','PEP','PFE','PFG','PG','PGR','PH','PHM','PKI','PLD','PNC','PNW','PPG','PPL','PRU','PSA','PX','QCOM','R','RAI','RF','RHI','RL','ROK','RRC','RTN','SBUX','SCHW','SE','SEE','SHW','SLB','SNA','SNDK','SO','SPG','SPLS','SRE','STI','STJ','STT','STZ','SWK','SYK','SYMC','SYY','T','TAP','TDC','TE','TGT','TIF','TJX','TMK','TMO','TROW','TRV','TSN','TSO','TSS','TWX','TXN','TXT','UNH','UNM','UNP','UPS','USB','UTX','VAR','VFC','VIAB','VLO','VMC','VNO','VRSN','VZ','WAT','WBA','WFC','WFM','WHR','WM','WMB','WMT','WU','WY','WYN','XEL','XL','XLNX','XOM','XRX','YHOO','YUM','ZBH','ZION']
jan = True
labels = ["1m","2m","3m","4m","5m","6m","7m","8m","9m","10m","11m","12m","24m","36m","const","Jan"]

## Recreate OG Jegadeesh Regression ##

In [6]:
st = dt.datetime(2009,1,1)
en = dt.datetime(2013,1,1)

newjeg_regr_coefs = []
for ticker in sp500_tickers_joey:
    try:
        data = web.get_data_yahoo(ticker, start=st, end=en)
    except:
        print(ticker, "error")
        continue
        
    mdata = data.resample("M",how='last').pct_change().dropna()
    Y = mdata["Adj Close"].values - np.mean(mdata["Adj Close"].values)
    if jan:
        X = np.zeros([len(mdata["Adj Close"].values),16]) # 1 constant + 12*mo + 1 2y + 1 3y + January
        X[0::12,-1] = 1
    else:
        X = np.zeros([len(mdata["Adj Close"].values),15]) # 1 constant + 12*mo + 1 2y + 1 3y
    
    # constant
    X[:,-2] = 1
    
    X = pd.DataFrame(X)
    
    # monthly lags
    for x in range(12):
        X[x] = pd.Series(mdata["Adj Close"]).shift(1+x).values
    
    # 24m lags
    X[12] = pd.Series(mdata["Adj Close"]).shift(24).values
    
    # 36m lags
    X[13] = pd.Series(mdata["Adj Close"]).shift(36).values
    
    # truncate
    X = X.ix[36:]
    Y = pd.DataFrame(Y).ix[36:]
        
    result = sm.OLS(Y,X).fit()
    #print(result.summary())
    newjeg_regr_coefs.append(result.params)
    
newjeg_avg_coefs = np.mean(np.array(newjeg_regr_coefs),axis=0)
for x in zip(labels, newjeg_avg_coefs):
    print(x)

('1m', -0.10363567641910768)
('2m', -0.048151123430096254)
('3m', -0.040262055154809621)
('4m', -0.05042889484470018)
('5m', -0.023210134278198125)
('6m', -0.037971654836054781)
('7m', -0.074416624952444194)
('8m', 0.038264835090824192)
('9m', 0.042407805478199112)
('10m', -0.042817087783895862)
('11m', 0.051075350659999665)
('12m', -0.061807162106087797)
('24m', -0.0074239523915772048)
('36m', 0.026073449680458813)
('const', -0.001419136853826451)
('Jan', 0.034390893455739244)


## Backtest Jegadeesh Regressions ##

In [48]:
# Out of Sample Data
dataset = sp500_tickers_joey[:]

# take list of stocks
st = dt.datetime(2009,1,1)
en = dt.datetime(2016,1,1)
#ours
#regr_coef = np.array([0.25298567255645865, -0.12886536167593118, 0.019203996926985011, 0.028682508989952348, -0.0066884204332014778, -0.10153536136157984, 0.025687100756060847, -0.010705398522536751, -0.032641524657629657, -0.042552908843307105, 0.0042183541030977626, -0.044232746890859702, -0.030952224152683578, -0.014879969341961664, 0.0028323358461219473, -0.0060362359252851142])
# jegadeesh
#regr_coef = np.array([-.0923,-.0073,.0208,.0154,.0148,.0205,.0087,.0065,.0178,.0151,.0224,.0339,.0171,.0187,-.0033,.0135])
#regr_coef = -np.zeros(16)
#regr_coef = np.zeros(16)
#regr_coef[0] = 1
#ours recent
regr_coef = np.array([-0.10060336951050865, -0.04646090585100697, -0.039148532450012707, -0.051681878472906742, -0.024573592050105345, -0.035499096200907201, -0.074405553384210912, 0.037950605435127974, 0.04182284042087446, -0.041514417764029844, 0.049731158178113521, -0.062613419313814575, -0.0065865733315576434, 0.024768369045317821, -0.0014942752035320926, 0.034550948063844261])

exp_ret = np.zeros([12*3,len(dataset)])
act_ret = np.zeros([12*3,len(dataset)])

t = 0

for ticker in dataset:
    # calculate returns    
    try:
        data = web.get_data_yahoo(ticker, start=st, end=en)
    except:
        print(ticker, "error")
        continue
    
    mdata = data.resample("M",how="last").pct_change()
    
    adj_close = mdata["Adj Close"]
      
    k = 0
    for i in range(len(adj_close)):
        if adj_close.index[i] > dt.datetime(2013,1,1):
            #print(adj_close.index[i])
            x_row = np.zeros(16) # 1-12 months, 24m, 36m, 1 constant, 1 january
            x_row[0:12] = adj_close[i-12:i].values[::-1]
            x_row[12] = adj_close[i-24]
            x_row[13] = adj_close[i-36]
            x_row[14] = 1
            if adj_close.index[i].month == 1:
                x_row[15] = 1
            exp_ret[k,t] = np.dot(x_row, regr_coef)
            act_ret[k,t] = adj_close[i]
            k += 1
    
    t += 1  # count ticker    

exp_ret_df = pd.DataFrame(data=exp_ret, index=pd.date_range(start=dt.datetime(2013,1,1), end=dt.datetime(2016,1,1),freq='M'),columns = dataset)
act_ret_df = pd.DataFrame(data=act_ret, index=pd.date_range(start=dt.datetime(2013,1,1), end=dt.datetime(2016,1,1),freq='M'),columns = dataset)
#print(exp_ret_df)

In [ ]:
dates = pd.date_range(start=dt.datetime(2013,1,1), end=dt.datetime(2016,1,1),freq='M')
frets = []
oldport = set()
turnover = []
for index, row in exp_ret_df.iterrows():
    newport = set()
    sorted_row = row.order(ascending=False)
    pret = 0
    for x in range(30):
        newport.add(sorted_row.index[x])
        #print("date", index,sorted_row.index[x],"expected",sorted_row[x],"actual",act_ret_df.loc[index,sorted_row.index[x]])
        pret += act_ret_df.loc[index,sorted_row.index[x]]
    print("new stocks",30 - len(newport & oldport))
    turnover.append((30 - len(newport & oldport))*2/float(30))
    oldport = newport
    pret = 1.0+pret/30.0
    frets.append(pret)
    #print("date", index)
    #print(sorted_row[:5].index.values)
    print("period return", pret)
    print("-------")
print("avg return", np.prod(frets)-1)
print(np.mean(turnover))

In [ ]:
plt.plot(np.cumprod(ourfrets)-1)
plt.plot(np.cumprod(jegfrets)-1)
plt.plot(np.cumprod(mktfrets)-1)
plt.title('Cumulative Returns')
plt.xlabel('Months after 1/1/2013')
plt.ylabel('Return')
plt.show()

## Industry Breakdown ##

In [9]:
import urllib.request as ul
from bs4 import BeautifulSoup

def scrape_list(site):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = ul.Request(site, headers=hdr)
    page = ul.urlopen(req)
    soup = BeautifulSoup(page,"html.parser")
    table = soup.find('table', {'class': 'wikitable sortable'})
    sector_tickers = dict()
    for row in table.findAll('tr'):
        col = row.findAll('td')
        if len(col) > 0 and (str(col[0].string.strip()) in sp500_tickers_joey):
            sector = str(col[3].string.strip()).lower().replace(' ', '_')
            ticker = str(col[0].string.strip())
            if sector not in sector_tickers:
                sector_tickers[sector] = list()
            sector_tickers[sector].append(ticker)
    return sector_tickers

SITE = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
industry_stocks = scrape_list(SITE)

In [11]:
st = dt.datetime(2009,1,1)
en = dt.datetime(2013,1,1)
industry_coefs = {}
jan = True

for industry in industry_stocks:
    regr_coefs = []
    k = 0

    for ticker in industry_stocks[industry]:
        try:
            data = web.get_data_yahoo(ticker, start=st, end=en)
            k += 1
        except:
            print(ticker, "error")
            continue
        
        mdata = data.resample("M",how='last').pct_change().dropna()
        
        if len(mdata) < 40:
            print(ticker, " not enough data")
            continue
        
        Y = mdata["Adj Close"].values - np.mean(mdata["Adj Close"].values)
        #print(industry, ticker, len(Y))
        if jan:
            X = np.zeros([len(mdata["Adj Close"].values),16]) # 1 constant + 12*mo + 1 2y + 1 3y + January
            X[0::12,-1] = 1
        else:
            X = np.zeros([len(mdata["Adj Close"].values),15]) # 1 constant + 12*mo + 1 2y + 1 3y

        # constant
        X[:,-2] = 1

        X = pd.DataFrame(X)

        # monthly lags
        for x in range(12):
            X[x] = pd.Series(mdata["Adj Close"]).shift(1+x).values

        # 24m lags
        X[12] = pd.Series(mdata["Adj Close"]).shift(24).values

        # 36m lags
        X[13] = pd.Series(mdata["Adj Close"]).shift(36).values

        # truncate
        X = X.ix[36:]
        Y = pd.DataFrame(Y).ix[36:]

        result = sm.OLS(Y,X).fit()
        #print(result.summary())
        regr_coefs.append(result.params)

    industry_coefs[industry] = np.mean(np.array(regr_coefs),axis=0)

In [13]:
pd.DataFrame(industry_coefs, index=labels)

,consumer_discretionary,consumer_staples,energy,financials,health_care,industrials,information_technology,materials,telecommunications_services,utilities
1m,-0.122529,-0.049655,-0.027743,-0.067222,-0.191282,-0.108873,-0.140392,-0.069615,-0.015543,-0.013374
2m,-0.088283,-0.092016,-0.160886,-0.066075,-0.103599,-0.062476,-0.057846,-0.101619,0.135735,-0.202466
3m,-0.047333,-0.098279,-0.096211,-0.145483,0.020549,-0.027965,-0.054860,-0.053924,-0.312772,0.075212
4m,-0.116657,-0.120636,-0.090582,-0.124854,-0.090401,-0.038980,-0.144465,-0.099183,-0.297295,-0.163565
5m,-0.020933,-0.122263,-0.032756,0.094938,0.010182,-0.046308,0.147573,-0.031686,-0.077924,-0.276313
6m,0.020634,-0.268077,0.075755,0.030577,-0.148595,-0.068872,-0.022040,-0.042167,-0.023310,0.082561
7m,-0.152202,-0.008220,-0.053009,-0.138984,-0.032322,-0.144431,-0.261874,-0.051569,0.025139,0.186448
8m,-0.019643,0.105194,0.141242,0.147445,-0.002676,0.138162,0.089352,0.179972,-0.341017,0.058693
9m,0.007542,-0.106662,0.161075,-0.051511,-0.045100,0.036424,-0.002103,-0.071733,-0.063953,-0.034224
10m,0.076281,0.109112,0.024557,0.029683,0.075891,0.157367,0.134757,0.134178,-0.123479,0.002811


## Vol Breakdown ##

In [20]:
st = dt.datetime(2009,1,1)
en = dt.datetime(2013,1,1)

# create deciles
stockvol_list = []
vol_dec = {"decile 1":[],
       "decile 2":[],
       "decile 3":[],
       "decile 4":[],
       "decile 5":[],
       "decile 6":[],
       "decile 7":[],
       "decile 8":[],
       "decile 9":[],
       "decile 10":[]}

for stock in sp500_tickers_joey:
    try:
        data = web.get_data_yahoo(stock, start=st, end=en)
    except:
        print(stock, "error")
        continue
        
    vol = np.std(data["Adj Close"].pct_change().dropna())
        
    #print(stock, vol)
    stockvol_list.append([stock, vol])

vol_sort = sorted(stockvol_list, key=lambda x: x[1], reverse=True)

i = 0
d = 1
while i < len(vol_sort):
    vol_dec["decile "+str(d)].append(vol_sort[i][0])
    i += 1
    if (i % int(len(vol_sort)/10.0))==0 and d != 10:
        d += 1

In [24]:
st = dt.datetime(2009,1,1)
en = dt.datetime(2013,1,1)
vol_dec_coefs = {}
jan = True

for d in vol_dec:
    regr_coefs = []
    k = 0

    for ticker in vol_dec[d]:
        try:
            data = web.get_data_yahoo(ticker, start=st, end=en)
            k += 1
        except:
            print(ticker, "error")
            continue
        
        mdata = data.resample("M",how='last').pct_change().dropna()
        
        if len(mdata) < 40:
            print(ticker, " not enough data")
            continue
        
        Y = mdata["Adj Close"].values - np.mean(mdata["Adj Close"].values)
        #print(d, ticker, len(Y))
        if jan:
            X = np.zeros([len(mdata["Adj Close"].values),16]) # 1 constant + 12*mo + 1 2y + 1 3y + January
            X[0::12,-1] = 1
        else:
            X = np.zeros([len(mdata["Adj Close"].values),15]) # 1 constant + 12*mo + 1 2y + 1 3y

        # constant
        X[:,-2] = 1

        X = pd.DataFrame(X)

        # monthly lags
        for x in range(12):
            X[x] = pd.Series(mdata["Adj Close"]).shift(1+x).values

        # 24m lags
        X[12] = pd.Series(mdata["Adj Close"]).shift(24).values

        # 36m lags
        X[13] = pd.Series(mdata["Adj Close"]).shift(36).values

        # truncate
        X = X.ix[36:]
        Y = pd.DataFrame(Y).ix[36:]
        #print("Ticker", ticker, "Regr. Start Date", mdata["Close"].index[36],"ndata",len(Y))

        result = sm.OLS(Y,X).fit()
        #print(result.summary())
        regr_coefs.append(result.params)

    #print("done")
    avg_coefs = np.mean(np.array(regr_coefs),axis=0)
    vol_dec_coefs[d] = avg_coefs
    
#vol_dec_data = dict(dec_coefs)

## Mega Backtester #

In [25]:
stock_industry_coefs = {}
stock_vol_coefs = {}
avg_regr_coef = np.array([-0.10060336951050865, -0.04646090585100697, -0.039148532450012707, -0.051681878472906742, -0.024573592050105345, -0.035499096200907201, -0.074405553384210912, 0.037950605435127974, 0.04182284042087446, -0.041514417764029844, 0.049731158178113521, -0.062613419313814575, -0.0065865733315576434, 0.024768369045317821, -0.0014942752035320926, 0.034550948063844261])

# coefs
vol_dec_coefs
industry_coefs

# mappings
industry_stocks
vol_dec 

for category in vol_dec:
    for stock in vol_dec[category]:
        stock_vol_coefs[stock] = vol_dec_coefs[category]
    
for category in industry_stocks:
    for stock in industry_stocks[category]:
        stock_industry_coefs[stock] = industry_coefs[category]

In [46]:
# Out of Sample Data
dataset = sp500_tickers_joey[:]

# take list of stocks
st = dt.datetime(2010,1,1)
en = dt.datetime(2016,1,1)

exp_ret = np.zeros([12*3,len(dataset)])
act_ret = np.zeros([12*3,len(dataset)])

t = 0

for ticker in dataset:
    # calculate returns
    # print(ticker)
    
    try:
        data = web.get_data_yahoo(ticker, start=st, end=en)
    except:
        print(ticker, "error")
        continue
    
    mdata = data.resample("M",how="last").pct_change()
    
    adj_close = mdata["Adj Close"]
      
    k = 0
    for i in range(len(adj_close)):
        if adj_close.index[i] > dt.datetime(2013,1,1):
            x_row = np.zeros(16) # 1-12 months, 24m, 36m, 1 constant, 1 january
            x_row[0:12] = adj_close[i-12:i].values[::-1]
            x_row[12] = adj_close[i-24]
            x_row[13] = adj_close[i-36]
            x_row[14] = 1
            if adj_close.index[i].month == 1:
                x_row[15] = 1
            
            exp_ret[k,t] = np.dot(x_row, .95*avg_regr_coef + .025*stock_industry_coefs[ticker] + .025*stock_vol_coefs[ticker])
            act_ret[k,t] = adj_close[i]
            #print("expected return",exp_ret[k,t],"actual return",act_ret[k,t])
            k += 1
    
    t += 1  # count ticker    

exp_ret_df = pd.DataFrame(data=exp_ret, index=pd.date_range(start=dt.datetime(2013,1,1), end=dt.datetime(2016,1,1),freq='M'),columns = dataset)
act_ret_df = pd.DataFrame(data=act_ret, index=pd.date_range(start=dt.datetime(2013,1,1), end=dt.datetime(2016,1,1),freq='M'),columns = dataset)

#exp_ret_df.to_csv("0-0-1.csv")
#act_ret_df.to_csv("act-ret.csv")
#print(exp_ret_df)

In [47]:
dates = pd.date_range(start=dt.datetime(2013,1,1), end=dt.datetime(2016,1,1),freq='M')
frets = []
oldport = set()
turnover = []
for index, row in exp_ret_df.iterrows():
    newport = set()
    sorted_row = row.order(ascending=False)
    pret = 0
    for x in range(30):
        newport.add(sorted_row.index[x])
        #print("date", index,sorted_row.index[x],"expected",sorted_row[x],"actual",act_ret_df.loc[index,sorted_row.index[x]])
        pret += act_ret_df.loc[index,sorted_row.index[x]]
    #print("new stocks",30 - len(newport & oldport))
    turnover.append((30 - len(newport & oldport))/float(30))
    oldport = newport
    pret = 1.0+pret/30.0
    frets.append(pret)
    #print("date", index)
    #print(sorted_row[:5].index.values)
    #print("period return", pret)
    #print("-------")
print("avg return", np.prod(frets)-1)
print("turnover",np.mean(turnover))

avg return 0.740792922967
turnover 0.82037037037


-c:7: FutureWarning: order is deprecated, use sort_values(...)


## Daily Data ##

In [50]:
st = dt.datetime(2009,1,1)
en = dt.datetime(2013,1,1)
dlabels = ["1d","2d","3d","4d","5d","6d","7d","8d","9d","10d","11d","12d","24d","36d","const","Jan"]

newjeg_regr_coefs = []
for ticker in sp500_tickers_joey:
    try:
        data = web.get_data_yahoo(ticker, start=st, end=en)
    except:
        print(ticker, "error")
        continue
        
    mdata = data.pct_change().dropna()
    Y = mdata["Adj Close"].values - np.mean(mdata["Adj Close"].values)
    if jan:
        X = np.zeros([len(mdata["Adj Close"].values),16]) # 1 constant + 12*mo + 1 2y + 1 3y + January
        X[0::12,-1] = 1
    else:
        X = np.zeros([len(mdata["Adj Close"].values),15]) # 1 constant + 12*mo + 1 2y + 1 3y
    
    # constant
    X[:,-2] = 1
    
    X = pd.DataFrame(X)
    
    # monthly lags
    for x in range(12):
        X[x] = pd.Series(mdata["Adj Close"]).shift(1+x).values
    
    # 24m lags
    X[12] = pd.Series(mdata["Adj Close"]).shift(24).values
    
    # 36m lags
    X[13] = pd.Series(mdata["Adj Close"]).shift(36).values
    
    # truncate
    X = X.ix[36:]
    Y = pd.DataFrame(Y).ix[36:]
        
    result = sm.OLS(Y,X).fit()
    #print(result.summary())
    newjeg_regr_coefs.append(result.params)
    
newjeg_avg_coefs = np.mean(np.array(newjeg_regr_coefs),axis=0)
for x in zip(dlabels, newjeg_avg_coefs):
    print(x)

('1d', -0.042857429702745101)
('2d', -0.0010708597791066813)
('3d', -0.040449226959345781)
('4d', 0.013552980039174465)
('5d', -0.027836391045316641)
('6d', -0.0023494877541811472)
('7d', -0.0070620488526828172)
('8d', -0.033849099121906777)
('9d', 0.0092494302928278812)
('10d', 0.03236396919556437)
('11d', -0.0072568592590411792)
('12d', -0.014274447921884417)
('24d', -0.011038582051959222)
('36d', -0.0030071571750894172)
('const', 0.00049178803680041455)
('Jan', -0.0014811175583305366)
